In [ ]:
# 머신러닝 모델 생성 및 예측, 시각화 스크립트의 클래스화

In [ ]:
class MLP():
    pass

In [ ]:
from keras import backend as K
from keras import layers as L
from keras.models import Model
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

def Make_model(train,val):
    K.clear_session()

    model_ckpt = ModelCheckpoint('model_ckpt.h5',save_best_only=True)
    reduce_lr = ReduceLROnPlateau(patience=8,verbose=1)
    early_stop = EarlyStopping(patience=10,verbose=2,monitor='loss')

    entry = L.Input(shape=(12, 12, 3))
    x = L.SeparableConv2D(256, (3,3), activation='relu')(entry)
    x = L.MaxPooling2D((2, 2))(x)
    x = L.SeparableConv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = L.MaxPooling2D((2, 2))(x)
    x = L.SeparableConv2D(1024, (2, 2), activation='relu', padding='same')(x)
    x = L.GlobalMaxPooling2D()(x)
    x = L.Dense(256)(x)
    x = L.ReLU()(x)
    x = L.Dense(64, kernel_regularizer=l2(2e-4))(x)
    x = L.ReLU()(x)
    x = L.Dense(27, activation='softmax')(x)

    model = Model(entry,x)
    model.summary()
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

    history = model.fit_generator(train,validation_data=val,epochs=600,
                                  callbacks=[model_ckpt,reduce_lr,early_stop],verbose=2)
    
    model.save('save_model.h5')
    model.save_weights('save_weight.h5')
    
    model_json = model.to_json()
    with open('model_json.json', 'w') as f:
        f.write(model_json)

    model_yaml=model.to_yaml()
    with open('model_json.yaml', 'w') as f:
        f.write(model_yaml)

    return history